In [ ]:
m1pr = _cmp.PowerLawGaussian_math(
    alpha=-alpha,
    min_pl=mmin,
    max_pl=mmax,
    lambda_g=lambda_peak,
    mean_g=mu_g,
    sigma_g=sigma_g,
    min_g=mmin,
    max_g=mu_g + 5 * sigma_g,
)
m2pr = _cmp.PowerLaw_math(alpha=beta, min_pl=mmin, max_pl=m1pr.maximum)

# Smooth the lower end of these distributions
dist = {
    "mass_1": _cmp.SmoothedProb(origin_prob=m1pr, bottom=mmin, bottom_smooth=delta_m),
    "mass_2": _cmp.SmoothedProb(origin_prob=m2pr, bottom=mmin, bottom_smooth=delta_m),
}


In [ ]:
class log_madau_rate(object):
    def __init__(self, gamma, kappa, zp):
        self.gamma = gamma
        self.kappa = kappa
        self.zp = zp

    def __call__(self, z):
        return (
            _np.log1p(_np.power(1 + self.zp, -self.gamma - self.kappa))
            + self.gamma * _np.log1p(z)
            - _np.log1p(_np.power((1 + z) / (1 + self.zp), self.gamma + self.kappa))
        )

In [ ]:
class redshift_prior(object):

    def log_prob(self, z_vals):

            gamma = dic_param["gamma"]
            zp = dic_param["zp"]
            kappa = dic_param["kappa"]
        self.log_rate_eval = log_madau_rate(gamma, kappa, zp)
        return (
            _np.log(self.cosmo.dVc_by_dz(z_vals))
            - _np.log1p(z_vals)
            + self.log_rate_eval(z_vals)
            
        )